<a href="https://colab.research.google.com/github/SudharsunRavi/VigiLens/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe opencv-python deepface

In [2]:
# Cell 2: Import after runtime restart
import cv2
import mediapipe as mp
import os
import numpy as np
from deepface import DeepFace

25-09-17 07:00:56 - Directory /root/.deepface has been created
25-09-17 07:00:56 - Directory /root/.deepface/weights has been created


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
mp_face_detection = mp.solutions.face_detection

def detect_faces(image):
    """Return bounding boxes of faces using MediaPipe"""
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as detector:
        results = detector.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        boxes = []
        if results.detections:
            for det in results.detections:
                bboxC = det.location_data.relative_bounding_box
                ih, iw, _ = image.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                boxes.append((x, y, w, h))
        return boxes

def get_face_embedding(image, bbox):
    """Crop face and get embedding using DeepFace"""
    x, y, w, h = bbox
    face_img = image[y:y+h, x:x+w]
    if face_img.size == 0:
        return None
    embedding = DeepFace.represent(face_img, model_name="Facenet", enforce_detection=False)[0]["embedding"]
    return np.array(embedding)

In [5]:
known_faces_dir = "/content/drive/MyDrive/Faces"

known_face_embeddings = []
known_face_names = []

for filename in os.listdir(known_faces_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        path = os.path.join(known_faces_dir, filename)
        name = os.path.splitext(filename)[0]

        image = cv2.imread(path)
        # Detect faces
        boxes = detect_faces(image)
        if len(boxes) == 0:
            print(f"[WARNING] No face detected in {filename}")
            continue

        # Use first detected face
        embedding = get_face_embedding(image, boxes[0])
        if embedding is not None:
            known_face_embeddings.append(embedding)
            known_face_names.append(name)
            print(f"[INFO] Encoded {name}")
        else:
            print(f"[WARNING] Could not generate embedding for {filename}")

# Save embeddings for later
np.save("/content/drive/MyDrive/Faces/embeddings.npy", np.array(known_face_embeddings))
np.save("/content/drive/MyDrive/Faces/names.npy", np.array(known_face_names))
print("[INFO] Saved embeddings and names")


25-09-17 07:04:18 - 🔗 facenet_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5 to /root/.deepface/weights/facenet_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:00<00:00, 115MB/s]


[INFO] Encoded Jai
[INFO] Encoded Mariraj
[INFO] Encoded Livingston
[INFO] Encoded Divya P
[INFO] Encoded Gowri
[INFO] Encoded Sivakumar
[INFO] Encoded Cyril
[INFO] Encoded Kannan
[INFO] Encoded Jamuna
[INFO] Encoded Prawin 
[INFO] Encoded Abinaya
[INFO] Encoded Sri Hari
[INFO] Encoded Anusha V
[INFO] Encoded Ebineshar
[INFO] Encoded Thamizhmaran
[INFO] Encoded Vijaypandi
[INFO] Encoded Chandragrace
[INFO] Encoded Kalai
[INFO] Encoded vijay
[INFO] Encoded Poovarasan
[INFO] Encoded Bavadharani
[INFO] Encoded Bharath
[INFO] Encoded Ranjith
[INFO] Encoded Ramya
[INFO] Encoded Mahendran
[INFO] Encoded Iyappan
[INFO] Encoded Venkat
[INFO] Encoded Palani
[INFO] Encoded Vijay D
[INFO] Encoded Vinoth
[INFO] Encoded Haja
[INFO] Encoded Suresh
[INFO] Encoded Deepa
[INFO] Encoded Divya Selvarai
[INFO] Encoded Mahalakshmi
[INFO] Encoded Om Ganesh
[INFO] Encoded Alex
[INFO] Encoded Deepth
[INFO] Encoded Thamotharan
[INFO] Encoded Manju
[INFO] Encoded Rajalingam
[INFO] Encoded Edward
[INFO] Encoded 

In [6]:
known_face_embeddings = np.load("/content/drive/MyDrive/Faces/embeddings.npy", allow_pickle=True)
known_face_names = np.load("/content/drive/MyDrive/Faces/names.npy", allow_pickle=True)
print(f"[INFO] Loaded {len(known_face_names)} embeddings")


[INFO] Loaded 56 embeddings


In [9]:
input_video_path = "/content/drive/MyDrive/Faces/Videos/Input/3.mp4"
output_video_path = "/content/drive/MyDrive/Faces/Videos/Output/3.mp4"

input_movie = cv2.VideoCapture(input_video_path)
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(input_movie.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_movie.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = input_movie.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_movie = cv2.VideoWriter(output_video_path, fourcc, fps, (int(width), int(height)))

frame_number = 0

while True:
    ret, frame = input_movie.read()
    if not ret:
        break
    frame_number += 1

    boxes = detect_faces(frame)
    for bbox in boxes:
        embedding = get_face_embedding(frame, bbox)
        if embedding is None:
            name = "Unknown"
        else:
            # Compare with known embeddings
            distances = np.linalg.norm(np.array(known_face_embeddings) - embedding, axis=1)
            min_idx = np.argmin(distances)
            if distances[min_idx] < 0.6:  # threshold, tweak as needed
                name = known_face_names[min_idx]
            else:
                name = "Unknown"

        # Draw rectangle + label
        x, y, w, h = bbox
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.rectangle(frame, (x, y - 25), (x+w, y), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (x + 6, y - 6), font, 0.6, (255, 255, 255), 1)

    print(f"Processing frame {frame_number}/{length}")
    output_movie.write(frame)

input_movie.release()
output_movie.release()
cv2.destroyAllWindows()
print("[INFO] Video processing complete!")


Processing frame 1/61
Processing frame 2/61
Processing frame 3/61
Processing frame 4/61
Processing frame 5/61
Processing frame 6/61
Processing frame 7/61
Processing frame 8/61
Processing frame 9/61
Processing frame 10/61
Processing frame 11/61
Processing frame 12/61
Processing frame 13/61
Processing frame 14/61
Processing frame 15/61
Processing frame 16/61
Processing frame 17/61
Processing frame 18/61
Processing frame 19/61
Processing frame 20/61
Processing frame 21/61
Processing frame 22/61
Processing frame 23/61
Processing frame 24/61
Processing frame 25/61
Processing frame 26/61
Processing frame 27/61
Processing frame 28/61
Processing frame 29/61
Processing frame 30/61
Processing frame 31/61
Processing frame 32/61
Processing frame 33/61
Processing frame 34/61
Processing frame 35/61
Processing frame 36/61
Processing frame 37/61
Processing frame 38/61
Processing frame 39/61
Processing frame 40/61
Processing frame 41/61
Processing frame 42/61
Processing frame 43/61
Processing frame 44/